In [1]:
print('get qualified txs')
import sys
sys.path.append("../helper_functions")
import duneapi_utils as d
import google_bq_utils as bqu
import clickhouse_utils as ch
sys.path.pop()

import time

import numpy as np
import pandas as pd
import os

get qualified txs


In [2]:
trailing_days = -1 #Pulls Everything

table_name = 'dune_ovm1_daily_data'

In [3]:
# Run Dune
print('     dune runs')
days_param = d.generate_query_parameter(input=trailing_days,field_name='trailing_days',dtype='number')
dune_df = d.get_dune_data(query_id = 3587947, #https://dune.com/queries/3587947
    name = table_name,
    path = "outputs",
    performance="medium",
    params = [days_param],
    num_hours_to_rerun=365*24 #pull latest, unless a ton of time passed
)
dune_df['source'] = 'dune'
dune_df['dt'] = pd.to_datetime(dune_df['dt']).dt.tz_localize(None).dt.strftime('%Y-%m-%d %H:%M:%S')

2024-07-23 15:06:43.673 | INFO     | duneapi_utils:get_dune_data:58 - Results available at https://dune.com/queries/3587947?trailing_days=-1


     dune runs


2024-07-23 15:06:44.947 | INFO     | duneapi_utils:get_dune_data:105 - ✨ Results saved as outputs/dune_ovm1_daily_data.csv, with 142 rows and 37 columns.


In [4]:
display(dune_df.sample(5))
display(dune_df.dtypes)

,active_secs_per_day,avg_blob_gas_price_on_l2,avg_l1_fee_scalar,avg_l1_gas_price_on_l2,avg_l2_gas_price,base_fee_gwei,blockchain,calldata_bytes_l2_per_day,calldata_bytes_user_txs_l2_per_day,calldata_gas_l2_per_day,...,l2_gas_used_user_txs_per_day,l2_num_attr_deposit_txs_per_day,l2_num_success_txs_per_day,l2_num_txs_per_day,l2_num_user_deposit_txs_per_day,median_l2_eth_fees_per_tx,num_blocks,num_users_per_day,last_updated,source
108,86275,None,None,None,0.015,None,optimism_legacy_ovm1,4852513,4562681,31174552,...,None,0,35419,39634,623,0.000382,41044,2047,2024-07-23 19:02:08.958684,dune
64,86359,None,None,None,0.015,None,optimism_legacy_ovm1,8991301,8012217,60234268,...,None,0,42429,44417,2331,0.000586,46748,6848,2024-07-23 19:02:08.958684,dune
140,85997,None,None,None,0.015,None,optimism_legacy_ovm1,3011068,2857764,20193916,...,None,0,22111,23684,350,0.001408,24034,2009,2024-07-23 19:02:08.958684,dune
67,86062,None,None,None,0.015,None,optimism_legacy_ovm1,2569376,2508456,15735560,...,None,0,27260,29151,145,0.001102,29325,1287,2024-07-23 19:02:08.958684,dune
90,86008,None,None,None,0.015,None,optimism_legacy_ovm1,847248,613140,5589888,...,None,0,2590,2647,565,0.000365,3222,714,2024-07-23 19:02:08.958684,dune


active_secs_per_day                                     int64
avg_blob_gas_price_on_l2                               object
avg_l1_fee_scalar                                      object
avg_l1_gas_price_on_l2                                 object
avg_l2_gas_price                                      float64
base_fee_gwei                                          object
blockchain                                             object
calldata_bytes_l2_per_day                               int64
calldata_bytes_user_txs_l2_per_day                      int64
calldata_gas_l2_per_day                                 int64
calldata_gas_user_txs_l2_per_day                        int64
chain_id                                                int64
compressedtxsize_approx_l2_per_day                    float64
compressedtxsize_approx_user_txs_l2_per_day           float64
dt                                             datetime64[ns]
equivalent_l1_tx_fee                                   object
l1_contr

In [5]:
# #BQ Upload
# bqu.append_and_upsert_df_to_bq_table(dune_df, table_name, unique_keys = ['chain_id','dt','blockchain'])

In [6]:
ch.write_df_to_clickhouse(dune_df, table_name, if_exists='replace')

Existing data in table 'dune_ovm1_daily_data' has been deleted.
existing: ['active_secs_per_day', 'Nullable(Int64)', '', '', '', '', '\navg_blob_gas_price_on_l2', 'Nullable(String)', '', '', '', '', '\navg_l1_fee_scalar', 'Nullable(String)', '', '', '', '', '\navg_l1_gas_price_on_l2', 'Nullable(String)', '', '', '', '', '\navg_l2_gas_price', 'Nullable(String)', '', '', '', '', '\nbase_fee_gwei', 'Nullable(String)', '', '', '', '', '\nblockchain', 'Nullable(String)', '', '', '', '', '\ncalldata_bytes_l2_per_day', 'Nullable(Int64)', '', '', '', '', '\ncalldata_bytes_user_txs_l2_per_day', 'Nullable(Int64)', '', '', '', '', '\ncalldata_gas_l2_per_day', 'Nullable(Int64)', '', '', '', '', '\ncalldata_gas_user_txs_l2_per_day', 'Nullable(Int64)', '', '', '', '', '\nchain_id', 'Nullable(Int64)', '', '', '', '', '\ncompressedtxsize_approx_l2_per_day', 'Nullable(String)', '', '', '', '', '\ncompressedtxsize_approx_user_txs_l2_per_day', 'Nullable(String)', '', '', '', '', '\ndt', 'Nullable(DateTim

TypeError: Object of type Timestamp is not JSON serializable